In [4]:
import os, sys, pimms, pandas, warnings, urllib, datetime
sys.path.append('..')
import numpy as np
import neuropythy as ny
import matplotlib as mpl
import matplotlib.pyplot as plt
import hcpannot
%matplotlib inline
%load_ext autoreload
%autoreload 2

OSError: [Errno 30] Read-only file system: '/data'

In [3]:
# The cache directories:
cache_path       = '/data/crcns2021'
image_cache_path = f'{cache_path}/annot-images'
v123_cache_path  = f'{cache_path}/annot-v123'
csulc_cache_path = f'{cache_path}/annot-csulc'
trace_save_path = f'{cache_path}/results/data_branch/save'
labels_path = f'{cache_path}/results/labels'

import os
# If you aren't using /data
os.environ['HCPANNOT_LOAD_PATH'] = cache_path

In [ ]:
# Additional matplotlib preferences:
font_data = {'family': 'sans-serif',
             'sans-serif': ['HelveticaNeue', 'Helvetica', 'Arial'],
             'size': 10,
             'weight': 'light'}
mpl.rc('font', **font_data)
# we want relatively high-res images, especially when saving to disk.
mpl.rcParams['figure.dpi'] = 72 * 2
mpl.rcParams['savefig.dpi'] = 72 * 4